# Demonstration of GeoPandas

GeoPandas is a powerful Python Package for manipulating and plotting geographical data. In this notebook we will cover:

1. GeoPandas installation
1. Creating a GeoPandas DataFrame from a standard geographic file format.
1. Setting the crs (Coordinate Reference System).
1. Creating a map from the GeoPandas DataFrame.
1. Merging geographical areas.
1. Selecting small areas within a larger area.
1. Handling point data.
1. Converting geography between BNG (British National Grid Eastings and Northings) and Latitude/Longitude.
1. Creating cusomizable maps with MatPlotLib

For areas we most commonly use shape files. The frequently come as a Zip file with the shape file and other metadata files. The repository for this notebook (https://github.com/michaelallen1966/2010_geopandas) contains all the necessary files. UK government pages to download these files tend to move around! At the time of writing the full England shape file download may be found at:

https://geoportal.statistics.gov.uk/datasets/lower-layer-super-output-areas-december-2011-boundaries-ew-bsc

## Installation

It is recommended that you install the environment as instructed in the README for this GitHub repository. You may also install packages manually ina current environment, though this method risks contability problems with your current environment. To install manually:

```
conda install geopandas pysal
conda install -c conda-forge geoplot
pip install contextily mapclassify
```

Alternative save the following as a file titled `environment.yml`:

```
name: geopandas
channels:
  - defaults
  - conda-forge
dependencies:
  - geopandas
  - geoplot
  - jupyterlab
  - pysal
  - pip
  - pip:
    - contextily
    - mapclassify
```

Then, from the folder where that yml file is, type the following into a terminal/command line: 

`conda env create -f environment.yml`

And then activate the environment with:

`conda activate geopandas`

And then launch Jupyter Lab with:

`jupyter-lab`


## Import modules

In [ ]:
import contextily as ctx
import geopandas
import matplotlib.pyplot as plt
import pandas as pd

## Load LSOA shape file

The following will load the shape file of the South West of England LSOA (Lower Super Output Areas) into a GeoPandas DataFrame. Note that we can set the crs (Coordinate Reference System) when loading a GeoPandas DataFrame. EPSG:27700 is the crs to use when geography is in BNG (British National Grid Eastings and Northings).

* EPSG:27700 OSGB 1936 / British National Grid -- United Kingdom Ordnance Survey. Co-ordinates are in Eastings (X) and Northings (Y).
* EPSG:4326 WGS 84 -- WGS84 - World Geodetic System 1984. Co-ordinates are in Longitude (X) and Latitude (Y).

This file is a manually clipped section of the South West of England. In this notebook we will restrict it to Devon and Cornwall.

In [ ]:
# Import England LSAO Shape file (use 'zip://'' prefic for zip files)
filename = "zip://./data/south_west_lsoa.zip"
lsoa_gdf = geopandas.read_file(filename, crs='EPSG:27700')

A GeoPandas DataFrame has a 'geometry' column which contains the geographical details used for plotting and other geographic methos.

In [ ]:
lsoa_gdf.head(3)

We can create a simple lot using the `plot()` method for the Geopandas DataFrame.

In [ ]:
lsoa_gdf.plot(figsize=(8,8));

## Using Spatial Relationship methods to select data lower super output areas in Devon or Cornwall

Here we will find LSOAs that are within the counties of Devon and Cornwall.

We will start by loading a file of geography for counties and unitary authorities.

In [ ]:
# Import County boundary shape file
filename = "zip://./data/Counties_and_Unitary_Authorities__December_2017" + \
    "__Boundaries_UK-shp.zip"
counties_gdf = geopandas.read_file(filename)
counties_gdf = counties_gdf.set_crs(epsg=27700)

In [ ]:
filename

In [ ]:
counties_gdf.head()

We can see the boundries woth the `plot()` method.

In [ ]:
counties_gdf.plot(figsize=(8,8));

### Lambda functions

Take a break and watch a 6 minute video on Lambda functions:

https://youtu.be/25ovCm9jKfA

### Finding the Devon and Cornwall data

The first thing we need to do is to idenitify the rows in the DataFrame that are in Cornwall and Devon. We can't just use county names as we must also include the unitary authorities 'Plymouth' and 'Torbay'.

The best way to apply a 'by row' function in DataFrames is to use the `map()` method. This applies a given function to all rows in a DataFrame. To check whether a value is in a list we need to apply a short function. As the function is simple we can use a *lambda* function as shown below.

In [ ]:
# Identify rows that are Devon or Cornwall (use Pandas map lambda function)
devon_cornwall = \
    counties_gdf['ctyua17nm'].map(
        lambda x: x in ['Devon', 'Cornwall', 'Plymouth', 'Torbay'])

In [ ]:
devon_cornwall

In [ ]:
devon_cornwall.sum()

In [ ]:
# Add new column to DataFrame. This will be a booelan (True/False) column
counties_gdf['devon_cornwall'] = devon_cornwall

In [ ]:
counties_gdf.head()

In [ ]:
mask = counties_gdf['devon_cornwall'] == True

In [ ]:
dc = counties_gdf.loc[mask]

In [ ]:
dc

## Merging areas together

Now we are going to merge LSOAs by whetehr they are in Devon and Cornwall or not (we can merge on any column in order to combine areas). This will create a new DataFrame. To merge areas we use the `dissolve()` method.

https://geopandas.org/aggregation_with_dissolve.html

In [ ]:
merged_gdf = counties_gdf.dissolve(by='devon_cornwall')
merged_gdf

Now we select just the geometry Devon and Cornwall row, This has an index value of 'True'.

In [ ]:
dc_geometry = merged_gdf.loc[True].geometry

Jupyter will display the geometry of a single geometry zone:

In [ ]:
dc_geometry

### Spatial relationships

GeoPandas can perorm a range of spacial relational test such as those shown in the figure and table below (from https://en.wikipedia.org/wiki/Spatial_relation).

![](./images/spatial_fig.png)

![](./images/spatial_table.png)



To get all LSOA in Devon or Cornwall we need to apply a test for `overlaps` or `within` (as `within` will miss those LSOA that are share a boundary with the Devon and Cornwall region. In Pandas we use `|` for `or`.

In [ ]:
# Check for LSOA overlapping with, or completely within, Devon & Cornwall
mask = (lsoa_gdf.overlaps(dc_geometry)) | (lsoa_gdf.within(dc_geometry))

In [ ]:
mask

In [ ]:
# Create new Geopands DataFrame by applying mask
devon_cornwall_lsoa = lsoa_gdf.loc[mask]

In [ ]:
devon_cornwall_lsoa.head()

In [ ]:
# Show map of new DataFrame
devon_cornwall_lsoa.plot();

## Loading point data (hospitals) and selecting those in Devon and Cornwall.

We will load up a CSV file with hospital (acute stroke unit) data and create a geometry column from exisiting location data. This time our location data uses  Longitude and Latitute. For this we will use a different crs, namely EPSG:4326 (alo known as WGS84). This is a global reference system for Long/Lat. We will demonstrate an alternative way of setting crs, setting it *after* we have created the GeoPandas DataFrame. 

In [ ]:
# Load hospital data in GeoPandas DataFrame
hospitals_gdf = geopandas.read_file('./data/hosp_107.csv')
# Set geometry (note longitude comes first as GeoPandas expects x/y geometry)
hospitals_gdf.geometry = geopandas.points_from_xy(
        hospitals_gdf.long, hospitals_gdf.lat)
# Set crs for geometry, using epsg4326 for lat/long
hospitals_gdf = hospitals_gdf.set_crs(epsg=4326)
hospitals_gdf.head()

We now convert crs from Lat/long to the BNG (EPSG 27700) we have been using in the LSOA DataFrame.

In [ ]:
hospitals_gdf = hospitals_gdf.to_crs(epsg=27700)

Notice how the geometry column has changed:

In [ ]:
hospitals_gdf.head()

Identify hospitals `within` the Devon and Cornwall geometry zone we created earlier.

In [ ]:
hospitals_gdf.plot();

In [ ]:
mask = hospitals_gdf.within(dc_geometry)
dc_hospitals = hospitals_gdf.loc[mask]
dc_hospitals

## Bring in travel times to closest hospital

We will load data on travel times from each LSOA to their closest acute stroke unit. The DataFrame `merge` method is used to merge this into our GeoPandas DataFrame.

In [ ]:
# Import table of travel times from LSOA to closest stroke unit
travel_time = pd.read_csv('./data/lsoa_107_ivt.csv')

In [ ]:
travel_time.head()

In [ ]:
# Merge data
devon_cornwall_lsoa = devon_cornwall_lsoa.merge(
    travel_time, left_on='lsoa11nm', right_on='area', how='left')

In [ ]:
devon_cornwall_lsoa.head()

## Mapping data with MatPlotLib

Using MatPlotLib gives us mreo power than the GeoPandas DataFrame `plot` method.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10)) # Make max dimensions 10x10 inch
devon_cornwall_lsoa.plot(ax=ax, # Set which axes to use for plot (only one here)
                 column='time_to_thrombolysis_unit', # Column to apply colour
                 antialiased=False, # Avoids artifact boundry lines
                 edgecolor='face', # Make LSOA boundry same colour as area
                 vmin=0, # Manual scale min (remove to make automatic)
                 vmax=70, # Manual scale max (remove to make automatic)
                 cmap='inferno_r', # Colour map to use
                 # Adjust size of colourmap key, and add label
                 legend_kwds={'shrink':0.5, 'label':'Travel time (mins)'},
                 # Set to display legend
                 legend=True)
ax.set_axis_off() # Turn of axis linea dn numbers
plt.savefig('map.jpg', dpi=300) # Save figure
plt.show()

Repeat, but display hospitals as an extra plot using `ax` as axis. We will add hospital name (just postcode here), and use contexity to add a base map.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10)) # Make max dimensions 10x10 inch
# Plot travel times for each LSOA
devon_cornwall_lsoa.plot(ax=ax, # Set which axes to use for plot (only one here)
        column='time_to_thrombolysis_unit', # Column to apply colour
        # antialiased=False, # Avoids artifact boundry lines
        edgecolor='face', # Make LSOA boundry same colour as area
        linewidth=0.0,# Use linewidth=0 to hide boarder lines
        vmin=0, # Manual scale min (remove to make automatic)
        vmax=70, # Manual scale max (remove to make automatic)
        cmap='inferno_r', # Coloour map to use
        # Adjust size of colourmap key, and add label
        legend_kwds={'shrink':0.4, 'label':'Travel time (mins)'},
        # Set to display legend
        legend=True,
        # Set transparancy (to help reveal basemap)
        alpha = 0.70)

# Plot location of hospitals
dc_hospitals.plot(ax=ax, edgecolor='k', facecolor='w', markersize=200, 
                  marker='*')
# Add labels
for x, y, label in zip(
    dc_hospitals.geometry.x, dc_hospitals.geometry.y, dc_hospitals.hospital):
        ax.annotate(label, xy=(x, y), xytext=(8, 8), textcoords="offset points",
                    backgroundcolor="w", fontsize=8)
        
# Add base map (note that we specifiy thr same CRS as we are using)
# Use manual zoom to adjust level of detail of base map
ctx.add_basemap(ax, 
                crs='EPSG:27700',
                source=ctx.providers.OpenStreetMap.Mapnik,zoom=10)
    
ax.set_axis_off() # Turn of axis line numbers
ax.set_title('Travel time (minutes) to closest acute stroke unit')
# Adjust for printing
ax.margins(0)
ax.apply_aspect()
plt.subplots_adjust(left=0.01, right=1.0, bottom=0.0, top=1.0)
# Save figure
plt.savefig('map.jpg', dpi=300)
plt.show()

# For fun?
## Hospitals and train lines

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10)) # Make max dimensions 10x10 inch

# Add Devon & Cornwall LSOA
devon_cornwall_lsoa.plot(ax=ax, alpha=0.5)

# Plot location of hospitals
dc_hospitals.plot(ax=ax, edgecolor='k', facecolor='w', markersize=200, 
                  marker='*')
ctx.add_basemap(ax, 
                crs='EPSG:27700',
                source=ctx.providers.OpenRailwayMap,
                alpha=0.5)

plt.show()

## Combining elements

In [ ]:
fig, ax = plt.subplots(figsize=(12,12)) # Make max dimensions 10x10 inch

# Plot location of hospitals
dc_hospitals.plot(ax=ax, edgecolor='k', facecolor='r', markersize=300, 
                  marker='*')

# Plot Devon and Cornwall LSOA
devon_cornwall_lsoa.plot(ax=ax, alpha=0.1)

ctx.add_basemap(ax, 
                crs='EPSG:27700',
                source=ctx.providers.OpenStreetMap.Mapnik, alpha=0.8)

ctx.add_basemap(ax, 
                crs='EPSG:27700',
                source=ctx.providers.OpenRailwayMap, alpha=0.3)


plt.show()

## List of all map providers

In [ ]:
providers = {}

def get_providers(provider):
    if "url" in provider:
        providers[provider['name']] = provider
    else:
        for prov in provider.values():
            get_providers(prov)

get_providers(ctx.providers)

In [ ]:
providers